In [1]:
import findspark
findspark.init()
import pyspark
import os
import sys
from env_variable import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, coalesce, lit, year, month, dayofmonth, dayofyear, hour, udf, expr, to_date
from pyspark.sql.types import StringType, StructType, DoubleType, IntegerType, StructField

# Set up environment variables
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Initialize Spark session
spark = SparkSession.builder \
    .appName('Kafka Streaming to Cassandra') \
    .config('spark.default.parallelism', 100) \
    .config('spark.cassandra.connection.host', 'localhost') \
    .getOrCreate()

# Define schema for the incoming JSON
schema = StructType([
    StructField("vendorid", IntegerType(), True),
    StructField("tpep_pickup_datetime", StringType(), True),
    StructField("tpep_dropoff_datetime", StringType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("ratecodeid", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("pulocationid", IntegerType(), True),
    StructField("dolocationid", IntegerType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("congestion_surcharge", DoubleType(), True)
])

# Read streaming data from Kafka
streaming_data = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BROKER) \
    .option("subscribe", 'test1') \
    .option("startingOffsets", "earliest") \
    .load()
parsed_df = streaming_data.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

def process_passenger_statistics(batch_df, batch_id):
    passenger_df = batch_df.groupBy("passenger_count").count().withColumnRenamed("count", "trip_count")\
        .withColumnRenamed("passenger_count", "passenger_number")

    df_cassandra = spark.read.format("org.apache.spark.sql.cassandra") \
        .options(table="passenger_number_statistics", keyspace="statistics") \
        .load()

    updated_df = passenger_df.join(df_cassandra, "passenger_number", "full") \
        .withColumn("total_count", coalesce(col("trip_count"), lit(0)) + coalesce(col("count"), lit(0))) \
        .drop("trip_count").drop("count") \
        .withColumnRenamed("total_count", "count")

    updated_df = updated_df.withColumn("partition_key", lit(1))
    updated_df = updated_df.na.drop()
    updated_df = updated_df.withColumn("passenger_number", updated_df["passenger_number"].cast("int"))

    updated_df.write.format("org.apache.spark.sql.cassandra") \
        .mode("append") \
        .options(table="passenger_number_statistics", keyspace="statistics") \
        .save()
        
query_passenger = parsed_df.writeStream \
    .foreachBatch(process_passenger_statistics) \
    .outputMode("update") \
    .start()
    
query_passenger.awaitTermination()


StreamingQueryException: Query [id = 4246de88-4fbf-4fa8-839d-c6309e71bd15, runId = 09141d9b-ff4f-4904-9925-838d65f7f28d] terminated with exception: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "C:\Spark\spark-3.3.1-bin-hadoop3\python\lib\py4j-0.10.9.5-src.zip\py4j\clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "C:\Spark\spark-3.3.1-bin-hadoop3\python\pyspark\sql\utils.py", line 272, in call
    raise e
  File "C:\Spark\spark-3.3.1-bin-hadoop3\python\pyspark\sql\utils.py", line 269, in call
    self.func(DataFrame(jdf, self.session), batch_id)
  File "C:\Users\minhh\AppData\Local\Temp\ipykernel_35596\3008992714.py", line 72, in process_passenger_statistics
    .save()
  File "C:\Spark\spark-3.3.1-bin-hadoop3\python\pyspark\sql\readwriter.py", line 966, in save
    self._jwrite.save()
  File "C:\Spark\spark-3.3.1-bin-hadoop3\python\lib\py4j-0.10.9.5-src.zip\py4j\jav